## Embedding generation for similarity Search
- PhraseBERT Encoder

In [ ]:
!pip install sentence_transformers
import json
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import numpy as np
import pandas as pd
from tqdm import trange
import torch
from torch import nn

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# First, upload final_db.json to sph google account

In [ ]:
with open('/content/drive/MyDrive/final_db.json', "r") as f:
  db = json.load(f)
embedder = SentenceTransformer('whaleloops/phrase-bert')
embedder.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.41k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': None}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
# Test functionality
phrase_list = [ 'play an active role', 'participate actively', 'active lifestyle']
phrase_embs = embedder.encode(phrase_list)
[p1, p2, p3] = phrase_embs
for phrase, embedding in zip(phrase_list, phrase_embs):
    print("Phrase:", phrase)
    print("Embedding:", embedding)
    print("")

Phrase: play an active role
Embedding: [ 1.54211000e-01 -1.15030736e-01  5.38258731e-01 -1.79741442e-01
 -1.62241131e-01  3.72248381e-01  2.95792699e-01 -4.61129434e-02
 -8.64347398e-01 -6.17788017e-01 -1.75134465e-01  5.74924462e-02
 -2.18788072e-01  5.63108409e-03 -3.68635565e-01  7.18021914e-02
  5.94799578e-01  4.34412450e-01 -2.89002568e-01  3.74785811e-01
 -3.13803583e-01 -3.39902639e-01  8.27495635e-01  5.68040431e-01
 -7.45191157e-01 -5.24603546e-01  7.00036466e-01  1.25253904e+00
  5.19454300e-01 -2.90108144e-01  5.96354119e-02  4.49128896e-01
  5.42105079e-01  5.27167976e-01 -1.05127370e+00  1.18721366e-01
  1.29327130e+00 -3.70293856e-01  6.54373109e-01  6.54097021e-01
 -6.00314856e-01 -2.84758717e-01 -6.40242100e-01  6.96587861e-01
 -2.18524933e-01 -7.77644694e-01  5.51177800e-01  1.57698035e-01
  2.24073425e-01  1.04450144e-01 -1.39757597e+00  3.72865647e-01
  1.02148616e+00  6.28725469e-01 -3.86314511e-01  2.60244161e-01
 -4.63665694e-01 -4.06141996e-01 -5.58624089e-01 -6

In [ ]:
import torch
from torch import nn
phrase_list = [ 'play an active role', 'participate actively', 'active lifestyle']
phrase_embs = embedder.encode(phrase_list)
[p1, p2, p3] = phrase_embs

# Code for function starts here
cos_sim = nn.CosineSimilarity(dim=0)
def cos_similarity(em_1, em_2):
  return cos_sim(torch.tensor(em_1), torch.tensor(em_2))

print(f'The cosine similarity between phrase 1 and 2 is: {cos_sim( torch.tensor(p1), torch.tensor(p2))}')
print(f'The cosine similarity between phrase 1 and 3 is: {cos_sim( torch.tensor(p1), torch.tensor(p3))}')
print(f'The cosine similarity between phrase 2 and 3 is: {cos_sim( torch.tensor(p2), torch.tensor(p3))}')


The cosine similarity between phrase 1 and 2 is: 0.8142534494400024
The cosine similarity between phrase 1 and 3 is: 0.613030195236206
The cosine similarity between phrase 2 and 3 is: 0.5848934054374695


## Using Threadpool

In [128]:
tags_embeddings = [embedder.encode(tag).tolist() for tag in ['Italy',
  'Napoli',
  'Serie A',
  'Champions League',
  'Osimhen',
  'Kvaratskhelia']]
tags_embeddings


[[-0.10219791531562805,
  -0.996174156665802,
  -0.205307736992836,
  -0.5581074953079224,
  0.5905963778495789,
  0.6777372360229492,
  0.5205838084220886,
  1.5349631309509277,
  -0.6165261268615723,
  -0.20140571892261505,
  -0.8591826558113098,
  -0.21466423571109772,
  -0.9393215179443359,
  1.2985585927963257,
  0.09126374125480652,
  0.0841328501701355,
  -0.024403274059295654,
  0.850459098815918,
  -0.20688556134700775,
  -0.004289828706532717,
  0.5797780156135559,
  0.34848281741142273,
  -0.46928682923316956,
  0.8654556274414062,
  -0.7427839636802673,
  -1.2064834833145142,
  0.7534224390983582,
  -0.20837156474590302,
  -0.2127981036901474,
  -0.7501603960990906,
  0.34423843026161194,
  0.350260466337204,
  -0.6336807012557983,
  0.1820203810930252,
  0.48219791054725647,
  0.999262809753418,
  -0.8256893754005432,
  -0.38082805275917053,
  -1.0955270528793335,
  0.3915959894657135,
  0.015846524387598038,
  -0.15212760865688324,
  -0.4499615728855133,
  0.7585346102714

In [145]:
def get_embedding(article_pair):
    article_tags, article_id = article_pair
    # Embed each tag separately and ensure they are converted to float64
    tags_embeddings = [np.array(embedder.encode(tag), dtype=np.float64).tolist() for tag in article_tags]
    return article_id, tags_embeddings

def process_articles(db):
    results = {}  # Use a list to store all batches for simplicity in JSON output
    max_workers = 25
    batch_size = 100

    article_tags = []
    article_ids = []
    for article in db:
      article_tags.append(article['tags'])
      article_ids.append(article['id'])
    article_id_pairs = list(zip(article_tags, article_ids))


    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i in trange(0, len(article_id_pairs), batch_size):
            current_batch = article_id_pairs[i:i+batch_size]
            print(f"Starting batch processing for articles {i+1} to {min(i+batch_size, len(article_id_pairs))}")
            futures = {executor.submit(get_embedding, pair): pair for pair in current_batch}
            for future in as_completed(futures):
                article_id, tags = future.result()
                results[article_id] = tags
    return results


In [ ]:
len(db)

2008

In [146]:
%%time
new_tags = process_articles(db)

  0%|          | 0/21 [00:00<?, ?it/s]

Starting batch processing for articles 1 to 100


  5%|▍         | 1/21 [00:12<04:00, 12.04s/it]

Starting batch processing for articles 101 to 200


 10%|▉         | 2/21 [00:24<03:50, 12.11s/it]

...

Starting batch processing for articles 1901 to 2000


 95%|█████████▌| 20/21 [03:56<00:11, 11.91s/it]

Starting batch processing for articles 2001 to 2008


100%|██████████| 21/21 [03:57<00:00, 11.29s/it]

CPU times: user 4min 11s, sys: 1min 1s, total: 5min 12s
Wall time: 3min 57s


## Merging with final_db

In [147]:
assigned = 0
for article in db:
    id = article['id']
    # Explicitly convert embeddings back to float64 before assignment
    article['phBERT_tags_embeddings'] = [np.array(tag, dtype=np.float64).tolist() for tag in new_tags[id]]
    assigned += 1
if assigned == len(db):
    print("All tags have been assigned")
else:
    print("Not all tags have been assigned")

All tags have been assigned


In [148]:
def convert_embeddings_to_list(articles):
    for article in articles:
        if 'phrase_Bert_tags_embeddings' in article:
            embeddings = article['phrase_Bert_tags_embeddings']
            if isinstance(embeddings, list) and all(isinstance(item, list) for item in embeddings):
                # Skip if already a list of lists
                continue
            # Convert each array to a list with float64 elements
            article['phrase_Bert_tags_embeddings'] = [np.array(tag, dtype=np.float64).tolist() if isinstance(tag, np.ndarray) else tag for tag in embeddings]

convert_embeddings_to_list(db)

In [149]:
fakes = []
for el in db:
  for embed in el['phrase_Bert_tags_embeddings']:
    if type(embed)== np.ndarray and el['id'] not in fakes:
      fakes.append(el['id'])
print(len(fakes)==0)

True


In [150]:
with open('/content/drive/MyDrive/final_db_pb_tags.json', "w") as file:
    json.dump(db, file, indent=4)